In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=40,
    remove_stop_words=False
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7011.65it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4715.26it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2627065.83it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9787.89it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4823.75it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 626376.26it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 4275.79it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fc44b27d400>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fc46045e9b0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
padding_callback = mz.dataloader.callbacks.BasicPadding()
fixed_padding_callback = mz.dataloader.callbacks.BasicPadding(
    fixed_length_left=10,
    fixed_length_right=40
)

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=fixed_padding_callback
)

In [8]:
model = mz.models.ConvKNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ConvKNRM(
  (embedding): Embedding(30058, 300)
  (q_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): Tanh()
    )
  )
  (d_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))

In [9]:
optimizer = torch.optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10,
    scheduler=scheduler,
    clip_norm=10
)

In [10]:
trainer.run()

[Iter-255 Loss-0.639]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6379 - normalized_discounted_cumulative_gain@5(0.0): 0.6798 - mean_average_precision(0.0): 0.6478



[Iter-510 Loss-0.348]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6053 - normalized_discounted_cumulative_gain@5(0.0): 0.6647 - mean_average_precision(0.0): 0.6269



[Iter-765 Loss-0.132]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5669 - normalized_discounted_cumulative_gain@5(0.0): 0.6215 - mean_average_precision(0.0): 0.5789



[Iter-1020 Loss-0.053]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5976 - normalized_discounted_cumulative_gain@5(0.0): 0.6584 - mean_average_precision(0.0): 0.6205



[Iter-1275 Loss-0.026]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6058 - normalized_discounted_cumulative_gain@5(0.0): 0.6654 - mean_average_precision(0.0): 0.6266



[Iter-1530 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6122 - normalized_discounted_cumulative_gain@5(0.0): 0.6676 - mean_average_precision(0.0): 0.6277



[Iter-1785 Loss-0.012]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6053 - normalized_discounted_cumulative_gain@5(0.0): 0.6658 - mean_average_precision(0.0): 0.6255



[Iter-2040 Loss-0.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6066 - normalized_discounted_cumulative_gain@5(0.0): 0.6658 - mean_average_precision(0.0): 0.6254



[Iter-2295 Loss-0.010]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.606 - normalized_discounted_cumulative_gain@5(0.0): 0.6654 - mean_average_precision(0.0): 0.6249



[Iter-2550 Loss-0.008]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6066 - normalized_discounted_cumulative_gain@5(0.0): 0.6658 - mean_average_precision(0.0): 0.6254

